# Optimization Chapter v1.0

## Note
This notebook is meant to follow the Charity Notebook.  The Charity Notebook is the first attempt at creating the desired machine learning model, but only boasts an average 73% accuracy.  This notebook is the second attempt, aiming to optimize the model for better accuracy. Preprocessing and preparation of the data remains the same.

## Background
*The nonprofit foundation Alphabet Soup wants a tool that can help it select the applicants for funding with the best chance of success in their ventures. With your knowledge of machine learning and neural networks, you’ll use the features in the provided dataset to create a binary classifier that can predict whether applicants will be successful if funded by Alphabet Soup.*

*From Alphabet Soup’s business team, you have received a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. Within this dataset are a number of columns that capture metadata about each organization, such as:*

* **EIN and NAME** — Identification columns
* **APPLICATION_TYPE** — Alphabet Soup application type
* **AFFILIATION** — Affiliated sector of industry
* **CLASSIFICATION** — Government organization classification
* **USE_CASE** — Use case for funding
* **ORGANIZATION** — Organization type
* **STATUS** — Active status
* **INCOME_AMT** — Income classification
* **SPECIAL_CONSIDERATIONS** — Special considerations for application
* **ASK_AMT** — Funding amount requested
* **IS_SUCCESSFUL** — Was the money used effectively


## Preprocessing

In [29]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [30]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [31]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [32]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df.APPLICATION_TYPE.value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [33]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [34]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [35]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [36]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [37]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [38]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [39]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [40]:
# Define the optimized model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
optimized_hidden_nodes_layer1 = 100  # Increased neurons
optimized_hidden_nodes_layer2 = 50  # Increased neurons
optimized_hidden_nodes_layer3 = 20  # Additional layer
number_input_features = len(X_train[0])
optimized_nn = tf.keras.models.Sequential()

# First hidden layer
optimized_nn.add(
    tf.keras.layers.Dense(units=optimized_hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
optimized_nn.add(tf.keras.layers.Dense(units=optimized_hidden_nodes_layer2, activation="relu"))

# Third hidden layer (Newly added)
optimized_nn.add(tf.keras.layers.Dense(units=optimized_hidden_nodes_layer3, activation="relu"))

# Output layer
optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
optimized_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Check the structure of the model
optimized_nn.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               4400      
                                                                 
 dense_9 (Dense)             (None, 50)                5050      
                                                                 
 dense_10 (Dense)            (None, 20)                1020      
                                                                 
 dense_11 (Dense)            (None, 1)                 21        
                                                                 
Total params: 10491 (40.98 KB)
Trainable params: 10491 (40.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# Train the optimized model
# Increased the epochs to 150 and set a batch size of 32
optimized_fit_model = optimized_nn.fit(X_train_scaled, y_train, epochs=150, batch_size=32)


Epoch 1/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5689 - accuracy: 0.7225
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5555 - accuracy: 0.7284
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5524 - accuracy: 0.7324
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5504 - accuracy: 0.7322
Epoch 5/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5494 - accuracy: 0.7333
Epoch 6/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7347
Epoch 7/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7347
Epoch 8/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5476 - accuracy: 0.7349
Epoch 9/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5462 - accuracy: 0.7345
Epoch 10/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accura

In [42]:
# Evaluate the optimized model using the test data
optimized_model_loss, optimized_model_accuracy = optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss: {optimized_model_loss}, Optimized Accuracy: {optimized_model_accuracy}")


268/268 - 0s - loss: 0.5687 - accuracy: 0.7259 - 288ms/epoch - 1ms/step
Optimized Loss: 0.5687263607978821, Optimized Accuracy: 0.7259474992752075


In [43]:
# Export our model to HDF5 file
optimized_nn.save("OptimizedAlphabetSoupCharity.h5")


c:\Users\WolfA\anaconda3\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Further Optimization

In [49]:
# Define the further optimized model
number_input_features = len(X_train[0])
further_optimized_nn = tf.keras.models.Sequential()

# First hidden layer with dropout
further_optimized_nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=number_input_features))
further_optimized_nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer with dropout
further_optimized_nn.add(tf.keras.layers.Dense(units=50, activation="relu"))
further_optimized_nn.add(tf.keras.layers.Dropout(0.2))

# Third hidden layer
further_optimized_nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Fourth hidden layer
further_optimized_nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Output layer
further_optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model with a learning rate of 0.001
opt = Adam(learning_rate=0.001)
further_optimized_nn.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Check the structure of the model
further_optimized_nn.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 100)               4400      
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_17 (Dense)            (None, 50)                5050      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_18 (Dense)            (None, 30)                1530      
                                                                 
 dense_19 (Dense)            (None, 20)                620       
                                                                 
 dense_20 (Dense)            (None, 1)                

In [50]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Compile the model with early stopping callbacks
further_optimized_nn.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


In [51]:
# Train the model with early stopping
further_optimized_fit_model = further_optimized_nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=200,  # Increased maximum epochs
    batch_size=32,
    callbacks=[early_stopping],  # Early stopping
    validation_data=(X_test_scaled, y_test)  # Validation data for early stopping
)

Epoch 1/200


804/804 [==============================] - 2s 2ms/step - loss: 0.5855 - accuracy: 0.7153 - val_loss: 0.5650 - val_accuracy: 0.7231
Epoch 2/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5644 - accuracy: 0.7285 - val_loss: 0.5604 - val_accuracy: 0.7250
Epoch 3/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5597 - accuracy: 0.7286 - val_loss: 0.5597 - val_accuracy: 0.7199
Epoch 4/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5567 - accuracy: 0.7292 - val_loss: 0.5544 - val_accuracy: 0.7255
Epoch 5/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7300 - val_loss: 0.5547 - val_accuracy: 0.7258
Epoch 6/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.7315 - val_loss: 0.5547 - val_accuracy: 0.7272
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.7303 - val_loss: 0.5528 - val_accuracy: 0.7280
Epoch 8/200

In [52]:
# Evaluate the further optimized model using the test data
further_optimized_model_loss, further_optimized_model_accuracy = further_optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Further Optimized Loss: {further_optimized_model_loss}, Further Optimized Accuracy: {further_optimized_model_accuracy}")

268/268 - 0s - loss: 0.5509 - accuracy: 0.7282 - 226ms/epoch - 844us/step
Further Optimized Loss: 0.5508613586425781, Further Optimized Accuracy: 0.7281632423400879


In [53]:
# Export our model to HDF5 file
further_optimized_nn.save("FurtherOptimizedAlphabetSoupCharity.h5")


## Final Optimization Attempt

In [67]:
# Define the final optimized model
number_input_features = len(X_train[0])
final_opti_nn = tf.keras.models.Sequential()

# First hidden layer with dropout
final_opti_nn.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=number_input_features))
# Alternative activation function: "tanh"
# final_opti_nn.add(tf.keras.layers.Dense(units=120, activation="tanh", input_dim=number_input_features))
final_opti_nn.add(tf.keras.layers.Dropout(0.3))

# Second hidden layer with dropout and batch normalization
# final_opti_nn.add(tf.keras.layers.Dense(units=60, activation="relu"))
# Alternative activation function: "swish"
final_opti_nn.add(tf.keras.layers.Dense(units=60, activation="swish"))
final_opti_nn.add(tf.keras.layers.BatchNormalization())
final_opti_nn.add(tf.keras.layers.Dropout(0.3))

# Third hidden layer
final_opti_nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Fourth hidden layer (New)
final_opti_nn.add(tf.keras.layers.Dense(units=15, activation="relu"))

# Output layer
final_opti_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
final_opti_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Check the structure of the model
final_opti_nn.summary()


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 120)               5280      
                                                                 
 dropout_12 (Dropout)        (None, 120)               0         
                                                                 
 dense_45 (Dense)            (None, 60)                7260      
                                                                 
 batch_normalization_4 (Bat  (None, 60)                240       
 chNormalization)                                                
                                                                 
 dropout_13 (Dropout)        (None, 60)                0         
                                                                 
 dense_46 (Dense)            (None, 30)                1830      
                                                      

In [68]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model with early stopping
final_opti_fit_model = final_opti_nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=200,
    batch_size=32,
    callbacks=[early_stopping],
    validation_data=(X_test_scaled, y_test)
)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5967 - accuracy: 0.7060 - val_loss: 0.5690 - val_accuracy: 0.7241
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5756 - accuracy: 0.7227 - val_loss: 0.5628 - val_accuracy: 0.7222
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5669 - accuracy: 0.7250 - val_loss: 0.5574 - val_accuracy: 0.7251
Epoch 4/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5634 - accuracy: 0.7287 - val_loss: 0.5571 - val_accuracy: 0.7252
Epoch 5/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5627 - accuracy: 0.7276 - val_loss: 0.5562 - val_accuracy: 0.7264
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5596 - accuracy: 0.7304 - val_loss: 0.5547 - val_accuracy: 0.7276
Epoch 7/200
804/804 [==============================] - 1s 2ms/step - loss: 0.5604 - accuracy: 0.7299 - val_loss: 0.5547 - val_accuracy: 0.7247

In [69]:
# Evaluate the final optimized model using the test data
final_opti_model_loss, final_opti_model_accuracy = final_opti_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Final Optimized Loss: {final_opti_model_loss}, Final Optimized Accuracy: {final_opti_model_accuracy}")


268/268 - 0s - loss: 0.5518 - accuracy: 0.7273 - 229ms/epoch - 854us/step
Final Optimized Loss: 0.5518247485160828, Final Optimized Accuracy: 0.7273469567298889


In [70]:
# Export our model to HDF5 file
final_opti_nn.save("FinalOptimizedAlphabetSoupCharity.h5")